### Chapter 14

만약에 메모리가 다 담을 수 없는 데이터를 다룬다고 하자. 가령 무한한 길이의 수열이라든지. 그러면 데이터를 전부 가져온 다음 필요한 처리를 하는 것은 불가능하다. 필요한 만큼씩만 가져와서 처리할 수 있는 기술이 필요하다. 이 역할을 하는 것이 바로 반복자, 제너레이터이다.

이를 처리하는 중요한 함수 중 하나는 `__iter__` 이다. 이는 반복자를 반환하는데, 이 함수를 구현하는 모든 객체는 '반복형'이다. 이때 반복자는 for문 등 다양한 곳에서 이미 암묵적으로 쓰이고 있다. 

In [1]:
s="ABCD"
for c in s:
  print(c)

A
B
C
D


위 코드를 반복자를 이용해서 다시 짜 보면 다음과 같다.


In [2]:
s="ABCD"
it=iter(s)
while True:
  try:
    print(next(it))
  except:
    del it
    break

A
B
C
D


반복자에 대한 표준 인터페이스는 `__next__` 와 `__iter__` 의 두 가지 메서드를 정의한다. 이 2가지 메서드가 정의되어 있으면 `__subclasshook__` 클래스 메서드에 의해 abc.Iterator 추상 클래스의 서브클래스로 분류되게 된다.

즉, 이 2가지 메서드만 구현하면 반복자로서의 기능을 할 수 있다는 뜻이다. 따라서 `next()` 를 호출하고 StopIteration 예외를 잡는 것 이외에는 항목이 소진되었는지 확인할 방법은 없다. 또한 반복자의 다른 위치로 돌아가는 함수도 없으므로 반복자는 재설장할 수 없다.

이러한 반복자를 이용하는 클래스를 한번 만들어 보자.


In [3]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:

  def __init__(self, text):
    self.text=text
    self.words=RE_WORD.findall(text)

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    return SentenceIterator(self.words)

class SentenceIterator:
  def __init__(self, words):
    self.words=words
    self.index=0

  def __next__(self):
    try:
      word=self.words[self.index]
    except:
      raise StopIteration()
    self.index+=1
    return word

  def __iter__(self):
    return self

st=Sentence("Me and Python")
for it in st:
  print(it)

Me
and
Python


이때 SentenceIterator 클래스를 따로 만든 것은 이유가 있다. 다중 반복을 지원해야 하기 때문이다. 즉 동일한 반복형 객체로부터 여러 독립적인 반복자를 가질 수 있어야 한다. 그걸 위해서는 iter() 를 호출할 때마다 독립적인 반복자가 만들어져야 하고, 따라서 클래스를 따로 만들 필요가 있었던 것이다.

위 코드를 제너레이터 함수를 이용하여 짤 수도 있다. 이 경우 코드가 매우 간단해진다.


In [4]:
import re
import reprlib

RE_WORD=re.compile('\w+')

class Sentence:

  def __init__(self, text):
    self.text=text
    self.words=RE_WORD.findall(text)

  def __repr__(self):
    return 'Sentence(%s)' % reprlib.repr(self.text)

  def __iter__(self):
    for word in self.words:
      yield word
    return

st=Sentence("Me and Python")
for it in st:
  print(it)

Me
and
Python


+ 제너레이터 함수

그럼 제너레이터 함수란 무엇인가? 본체 안에 `yield` 키워드를 가진 함수는 모두 제너레이터 함수다. 이는 호출되면 제너레이터 객체를 반환한다. 이 함수의 좋은 점은 느긋하게, 필요할 때만 계산한다는 점이다.

In [5]:
def gen_AB():
  print('start')
  yield 'A'
  print('continue')
  yield 'B'
  print('end')

for c in gen_AB():
  print('->', c)

start
-> A
continue
-> B
end


+ 제너레이터 표현식

지능형 리스트의 느긋한 계산 버전인 제너레이터 표현식도 존재한다. 필요에 따라 항목을 느긋하게 생성해 주는 것이다.



In [7]:
def gen_AB():
  print('start')
  yield 'A'
  print('continue')
  yield 'B'
  print('end')

res1=[x*3 for x in gen_AB()]
for i in res1: # 조급하게 계산됨
  print('->',i)

res2=(x*3 for x in gen_AB())
for i in res2: #느긋하게, 그 항목이 필요할 때 정확하게 계산됨
  print('->',i)



start
continue
end
-> AAA
-> BBB
start
-> AAA
continue
-> BBB
end


+ yield from 구문

다른 제너레이터에서 생성된 값을 상위 제너레이터 함수에서 생성해야 할 수 있다. 전통적인 중첩 for문 방식은 다음과 같다.



In [8]:
def chain(*iterables):
  for it in iterables:
    for i in it:
      yield i

s="abc"
t=(0,1,2)
list(chain(s,t))

['a', 'b', 'c', 0, 1, 2]

이때 yield from 구문을 사용하면 내부 for 루프를 완전히 대체할 수 있다.

In [9]:
def chain(*iterables):
  for it in iterables:
    yield from it

s="abc"
t=(0,1,2)
list(chain(s,t))

['a', 'b', 'c', 0, 1, 2]

훨씬 올바르고 가독성도 좋아졌다. 또한 yield from은 단순한 편리 구문이 아니라, 외부 제너레이터의 호출자와 내부 제너레이터의 연결 통로를 만든다. 제너레이터를 코루틴으로 사용하는 경우 이 점이 매우 중요해지는데, 코루틴에 대해 살펴보는 16장에서 더 공부하도록 한다.

+ iter() 함수

파이썬은 어떤 객체를 반복해야 할 때 iter(x)를 호출한다. 그러나 이는 반복자 생성을 위해 인수를 두 개 넣어 호출할 수도 있다. 그럴 땐 첫번째 인수는 값 생성을 위해 인수 없이 반복적으로 호출되는 콜러블, 두 번째 인수는 구분 표시 즉 앞의 콜러블에서 이 값이 반환되면 반복자가 예외를 발생시키도록 만든다.

1이 나올 때까지 육면체 주사위를 굴리는 코드는 다음과 같다.

In [21]:
import random

def d6():
  return random.randint(1,6)

d6_iter=iter(d6,1)
for roll in d6_iter:
  print(roll)

5
2
5
5
6
6
6
4
2
2
5
3
2
4


1이 구분 표시이기 때문에, 1이 나올 때까지의 주사위 횟수는 계속 달라지지만 1은 결코 출력되지 않는다. 1이 나오는 순간 예외가 발생해 for 루프가 종료되기 때문이다. 또한 d6_iter 객체는 소모된 후에는 쓸모가 없어짐에 주의하자. 반복자는 재설정할 수 없다.